In [1]:
configs = {"fs.azure.account.auth.type": "OAuth",
       "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
       "fs.azure.account.oauth2.client.id": "<app_id>",
       "fs.azure.account.oauth2.client.secret": "<password>",
       "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<tenant>/oauth2/token",
       "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

dbutils.fs.mount(
source = "abfss://<container_name>@<storage_acct_name>.dfs.core.windows.net/<folder>",
mount_point = "/mnt/ticketdata",
extra_configs = configs)

In [2]:
ticketDF = spark.read.format('csv').options(
  header='true', inferschema='true').load("/mnt/ticketdata/*/*/*.csv")

# read the parking ticket csv files and write the output to parquet format for easy querying.
ticketDF.write.mode("append").parquet("/mnt/ticketdata/parquet/tickets")
print("Done")

In [3]:
# print the number of tickets (from csv)
print("Number of tickets in the database: ", ticketDF.count())

In [4]:
#count number of tickets from Parquet data
parquetTicketDF = spark.read.format('parquet').options(
    inferschema='true').load("/mnt/ticketdata/parquet/tickets")

print("Number of tickets in the database: ", parquetTicketDF.count())

In [5]:
parquetTicketDF.show(20, False)

In [6]:
parquetTicketDF.createOrReplaceTempView('TicketTable')
agg1 = spark.sql("SELECT car_make, count(*) as n FROM TicketTable GROUP BY car_make ORDER BY n DESC")
print('Tickets by car make: ', agg1.show(100, False))

In [7]:
parquetTicketDF.createOrReplaceTempView('TicketTable')
agg1 = spark.sql("SELECT plate_number, count(*) as n FROM TicketTable GROUP BY plate_number ORDER BY n DESC")
print('Top Offenders: ', agg1.show(100, False))

In [8]:
parquetTicketDF.createOrReplaceTempView('TicketTable')
rs1 = spark.sql("SELECT * FROM TicketTable WHERE plate_number = '<license_plate>'")
rs1.show()